# A simple example of parallel computing with Dask on ARM data

In [ ]:
#imports
import os
import numpy as np
from matplotlib import pyplot as plt
import dask.array as da
import dask.bag as db
from dask.diagnostics import Profiler, ResourceProfiler, CacheProfiler

from dask_jobqueue import PBSCluster
from dask.distributed import Client, progress, metrics, LocalCluster

import xarray as xr
%matplotlib inline

In [ ]:
in_directory = '/lustre/or-hydra/cades-arm/proj-shared/data_transfer/sgp/sgpmetE13.b1/'

In [ ]:
all_files = os.listdir(in_directory)
all_files.sort()

In [ ]:
test_file_name = os.path.join(in_directory, all_files[23])
test_dataset = xr.open_dataset(test_file_name)

In [ ]:
my_figure = plt.figure(figsize=[10,5])
test_dataset.temp_mean.plot()

In [ ]:
my_mean = float(test_dataset.temp_mean.mean())
my_max = float(test_dataset.temp_mean.max())
my_min = float(test_dataset.temp_mean.min())

In [ ]:
print("Mean: {}, Max: {}, Min: {}".format(my_mean, my_max, my_min ))

In [ ]:
test_dataset.close()

In [ ]:
def analyze_this_days_temps(filename):
    this_dataset = xr.open_dataset(filename)
    my_mean = float(test_dataset.temp_mean.mean())
    my_max = float(test_dataset.temp_mean.max())
    my_min = float(test_dataset.temp_mean.min())
    this_dataset.close()
    return my_mean, my_max, my_min

In [ ]:
fqn_files = [os.path.join(in_directory,thisfile) for thisfile in all_files]

In [ ]:
analyze_this_days_temps(fqn_files[235])

In [ ]:
client = Client('arm-jupyter.ornl.gov:5555')

In [ ]:
client

In [19]:
future = client.map(analyze_this_days_temps, fqn_files)

In [20]:
progress(future)

VBox()

In [ ]:
my_data = client.gather(future)